In [1]:
from ast import literal_eval
import pandas as pd
from test import assess_performance

from IPython.core.display import HTML

In [2]:
df_tg = pd.read_csv("taggers_tagged_test.csv")
df_test = pd.read_csv("full_test_kw_lbl.csv")

In [3]:
cols = ["age_kw_pred", 'age']
for col in cols:
    df_test[col] = df_test[col].apply(lambda x: list(sorted(list(set(literal_eval(x))))))

In [4]:
infant = 'Infants/Toddlers (<5 years old)'
child = 'Children/Youth (5 to 17 years old)'
adult = 'Adult (18 to 59 years old)'
older = 'Older Persons (60+ years old)'

In [5]:
def tags_to_list(row):
    ret = list()
    if row[infant]:
        ret.append(infant)
    if row[child]:
        ret.append(child)
    if row[adult]:
        ret.append(adult)
    if row[older]:
        ret.append(older)
    return ret

df_tg["age_tg"] = df_tg.apply(tags_to_list, axis=1)

In [8]:
df_test = df_test[df_test['entry_id'].isin(df_tg['entry_id'])]
df_aligned = pd.merge(df_test[['entry_id', 'age_kw_pred', 'age']], df_tg[['entry_id', 'age_tg']])
df_aligned.shape

(250, 4)

In [9]:
# see the agreement btw the original lbls and the new lbls
assess_performance(df_aligned['age'], df_aligned["age_tg"], list(sorted([infant, child, adult, older])))

,macro_precision,macro_recall,macro_f1_score,1_precision,0_precision,1_recall,0_recall,1_f1_score,0_f1_score,hamming_loss,zero_one_loss
Adult (18 to 59 years old),0.550,0.64800,0.55100,0.133,0.96600,0.46200,0.83500,0.2070,0.896,0.184,0.184
Children/Youth (5 to 17 years old),0.692,0.72500,0.69400,0.519,0.86600,0.74700,0.70300,0.6120,0.776,0.284,0.284
Infants/Toddlers (<5 years old),0.689,0.64900,0.66600,0.429,0.94900,0.33300,0.96600,0.3750,0.957,0.080,0.080
Older Persons (60+ years old),0.701,0.92500,0.76400,0.407,0.99600,0.91700,0.93300,0.5640,0.963,0.068,0.068
mean,0.658,0.73675,0.66875,0.372,0.94425,0.61475,0.85925,0.4395,0.898,0.154,0.154


In [10]:
df_res = assess_performance(df_aligned['age_kw_pred'], df_aligned["age_tg"], list(sorted([infant, child, adult, older])))
df_res.to_excel("Keyword-based Age Labeling Results.xlsx", index=True)
df_res

,macro_precision,macro_recall,macro_f1_score,1_precision,0_precision,1_recall,0_recall,1_f1_score,0_f1_score,hamming_loss,zero_one_loss
Adult (18 to 59 years old),0.560,0.77100,0.523,0.13300,0.98800,0.84600,0.6960,0.22900,0.8170,0.296,0.296
Children/Youth (5 to 17 years old),0.844,0.88600,0.857,0.72600,0.96100,0.92000,0.8510,0.81200,0.9030,0.128,0.128
Infants/Toddlers (<5 years old),0.812,0.71400,0.752,0.66700,0.95800,0.44400,0.9830,0.53300,0.9700,0.056,0.056
Older Persons (60+ years old),0.912,0.91200,0.912,0.83300,0.99200,0.83300,0.9920,0.83300,0.9920,0.016,0.016
mean,0.782,0.82075,0.761,0.58975,0.97475,0.76075,0.8805,0.60175,0.9205,0.124,0.124


In [16]:
display(HTML(df_aligned.to_html()))

,entry_id,age_kw_pred,age_tg
0,186169,[Children/Youth (5 to 17 years old)],[Children/Youth (5 to 17 years old)]
1,489394,[],[]
2,497984,[],[]
3,489948,"[Adult (18 to 59 years old), Children/Youth (5 to 17 years old)]",[Children/Youth (5 to 17 years old)]
4,184109,"[Adult (18 to 59 years old), Children/Youth (5 to 17 years old)]",[Children/Youth (5 to 17 years old)]
5,183984,[],[Children/Youth (5 to 17 years old)]
6,183806,"[Adult (18 to 59 years old), Children/Youth (5 to 17 years old)]",[Children/Youth (5 to 17 years old)]
7,489965,[Adult (18 to 59 years old)],[]
8,190209,[],[]
9,190266,[],[]
